# Filter Latest Tweets 
tweets that are only written in ID in their latest tweet

In [ ]:
import os
import re
import glob
import json
import pickle

from tqdm import tqdm
import datetime as dt
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import langdetect

In [ ]:
pd.set_option("max_columns", 55)
pd.set_option("max_rows", 20)

In [ ]:
# path input
CHECK_LATEST_TWEET_PATH = "../data/supports/check_latest_tweet.pkl"
PROFILE_W_LANGUAGE_PATH = "../data/supports/profile_w_lang.json"

# path output
PROFILE_DESCRIPTION_PATH = "../data/supports/profile_description.json"
PROFILE_ID_PATH = "profile_id.json"
SCREEN_NAME_ID_PATH = "../data/supports/screen_name_id.json"

In [ ]:
d_profile = pickle.load(open(CHECK_LATEST_TWEET_PATH, 'rb'))

In [ ]:
d_profile.shape

In [ ]:
d_profile.columns

## Filtering

In [ ]:
## duration latest tweet of an account to the date when the data gathered
d_profile = d_profile[d_profile["duration_latest_tweet"] < 7]

In [ ]:
d_profile[d_profile.description.str.len() > 1].shape

In [ ]:
d_profile_desc = d_profile[d_profile.description.str.len() > 1]

In [ ]:
d_profile_desc[["screen_name", "name", "description"]].to_json(
    PROFILE_DESCRIPTION_PATH, orient='records')

#### filter akun dengan bahasa id

In [ ]:
d_profile_w_lang = pd.read_json(PROFILE_W_LANGUAGE_PATH)

In [ ]:
d_profile_w_lang.head()

In [ ]:
d_profile_w_lang = d_profile_w_lang[["screen_name", "lang_desc"]]

In [ ]:
## merge label language dengan profile description
d_profile_desc = pd.merge(d_profile_desc, d_profile_w_lang, on="screen_name")

## filter lang_desc == id
d_profile_id = d_profile_desc[d_profile_desc["lang_desc"]=="id"]

## save ke file
d_profile_id.to_json(PROFILE_ID_PATH, orient="records")

## crunching

In [ ]:
with open(PROFILE_ID_PATH, 'r', encoding='utf-8') as f:
    data_dict = json.load(f)

In [ ]:
d_profile = pd.DataFrame(data_dict)

In [ ]:
d_profile.columns

In [ ]:
cols = ['screen_name', 'name', 'description', 'friends_count', 'followers_count', 'days_duration', 
        'activity_count', 'activity_perday', 'blocked_by', 'blocking',
        'contributors_enabled', 'created_at', 'created_at_format',
       'default_profile', 'default_profile_image',
       'duration_latest_tweet', 'entities', 'favourites_count',
       'follow_request_sent', 'following',
       'geo_enabled', 'has_extended_profile', 'id', 'id_str',
       'is_translation_enabled', 'is_translator', 'lang', 'lang_desc',
       'latest_tweet', 'listed_count', 'live_following', 'location', 'muting',
       'notifications', 'profile_background_color',
       'profile_background_image_url', 'profile_background_image_url_https',
       'profile_background_tile', 'profile_banner_url', 'profile_image_url',
       'profile_image_url_https', 'profile_link_color', 'profile_location',
       'profile_sidebar_border_color', 'profile_sidebar_fill_color',
       'profile_text_color', 'profile_use_background_image', 'protected',
       'status', 'statuses_count', 'time_zone',
       'translator_type', 'url', 'utc_offset', 'verified',
       'withheld_in_countries']

In [ ]:
d_profile = d_profile[cols]

In [ ]:
d_profile.head()

dump to file

In [ ]:
d_profile.screen_name.to_json(SCREEN_NAME_ID_PATH, orient='records')

### fetch to file

In [ ]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, np.bool_):
            return str(obj)
        else:
            return super(NpEncoder, self).default(obj)

In [ ]:
def write_json(filename, data):
    path = os.path.join('../data/profile', filename+'json')
    if not os.path.exists(path):
        with open(path, 'w') as f:
            f.write(json.dumps(data, cls=NpEncoder))

In [ ]:
for index in tqdm(d_profile.index):
    profile_dict = d_profile.loc[index].to_dict()
    write_json(profile_dict["screen_name"], profile_dict)